# Parallel Tempering with THRML-Boost

Standard block Gibbs sampling is fast, but it has a blind spot: **multimodal distributions**. When the energy landscape has multiple deep wells separated by high barriers, the sampler gets trapped in whichever mode it finds first and never escapes.

This notebook shows the problem concretely, then demonstrates how parallel tempering fixes it — and how THRML-Boost runs all chains simultaneously via `jax.vmap` so you pay no extra compile cost for additional chains.

In [ ]:
import time
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [ ]:
from thrml_boost.block_management import Block
from thrml_boost.block_sampling import SamplingSchedule, sample_states
from thrml_boost.models.ising import IsingEBM, IsingSamplingProgram, hinton_init
from thrml_boost.pgm import SpinNode
from thrml_boost.tempering import parallel_tempering

## Setup: a strongly ferromagnetic Ising model

We use a 10×10 periodic Ising grid with strong ferromagnetic coupling and no bias. This model has two symmetric ground states: all spins up and all spins down, separated by a large energy barrier. It's a clean testbed for mixing.

In [ ]:
side = 10
G = nx.grid_2d_graph(side, side, periodic=True)
coord_to_node = {coord: SpinNode() for coord in G.nodes()}
nx.relabel_nodes(G, coord_to_node, copy=False)

nodes = list(G.nodes())
edges = list(G.edges())

beta_target = 2.0
biases = jnp.zeros(len(nodes))
weights = jnp.ones(len(edges)) * 1.0  # ferromagnetic

# Two-color for block Gibbs
coloring = nx.bipartite.color(G)
color0 = [n for n, c in coloring.items() if c == 0]
color1 = [n for n, c in coloring.items() if c == 1]
free_blocks = [Block(color0), Block(color1)]

key = jax.random.key(42)
print(f"Nodes: {len(nodes)}, Edges: {len(edges)}, Block sizes: {len(color0)}, {len(color1)}")

## The problem: Gibbs gets stuck

We track the **mean magnetization** $\langle s \rangle = \frac{1}{N}\sum_i s_i$ over time. A sampler that mixes well should oscillate between $+1$ (all up) and $-1$ (all down). A stuck sampler stays near one value indefinitely.

In [ ]:
ebm = IsingEBM(nodes, edges, biases, weights, jnp.array(beta_target))
program = IsingSamplingProgram(ebm, free_blocks, clamped_blocks=[])

key, subkey = jax.random.split(key)
init_state = hinton_init(subkey, ebm, free_blocks, ())

schedule = SamplingSchedule(n_warmup=50, n_samples=400, steps_per_sample=2)
key, subkey = jax.random.split(key)
gibbs_samples = sample_states(subkey, program, schedule, init_state, [], [Block(nodes)])

spins = 2 * gibbs_samples[0].astype(jnp.int8) - 1
mag_gibbs = jnp.mean(spins, axis=-1)

plt.figure(figsize=(12, 3))
plt.plot(mag_gibbs, lw=0.8, color="steelblue")
plt.axhline(0, color="k", lw=0.5, linestyle="--")
plt.fill_between(range(len(mag_gibbs)), -1, 1, alpha=0.03, color="gray")
plt.ylim(-1.1, 1.1)
plt.xlabel("Sample")
plt.ylabel("Mean magnetization")
plt.title(f"Standard Gibbs at β={beta_target} — trapped in one mode")
plt.tight_layout()
plt.show()

## The fix: parallel tempering

Parallel tempering runs multiple chains at different temperatures simultaneously:

- **Hot chains** (small β) — energy barriers are flattened, the chain explores freely
- **Cold chains** (large β) — accurate samples from the target distribution
- Every round, adjacent chains **propose a swap**. If accepted, the cold chain receives a configuration the hot chain found on the other side of a barrier.

But this only works if the temperature ladder is tuned correctly. If adjacent chains are too different, swaps are almost never accepted and the chains don't communicate. Let's see what that looks like.

In [ ]:
# A naive beta ladder — large gap at the bottom, crowded at the top
betas_naive = [0.1, 0.3, 0.7, 1.2, beta_target]
n_chains = len(betas_naive)

ebms_naive = [IsingEBM(nodes, edges, biases, weights, jnp.array(b)) for b in betas_naive]
programs_naive = [IsingSamplingProgram(e, free_blocks, []) for e in ebms_naive]

key, subkey = jax.random.split(key)
init = hinton_init(subkey, ebms_naive[0], free_blocks, ())


@jax.jit
def one_round_naive(key, states):
    return parallel_tempering(
        key,
        ebms_naive,
        programs_naive,
        states,
        [],
        n_rounds=1,
        gibbs_steps_per_round=2,
    )


mag_naive = []
states_naive = [init] * n_chains
key, subkey = jax.random.split(key)
for k in jax.random.split(subkey, 400):
    states_naive, _, stats_naive = one_round_naive(k, states_naive)
    cold = jnp.concatenate(states_naive[-1])
    mag_naive.append(float(jnp.mean(2 * cold.astype(jnp.int8) - 1)))

print("Swap acceptance rates with naive ladder:")
for i, r in enumerate(stats_naive["acceptance_rate"]):
    flag = "✓" if 0.2 <= float(r) <= 0.5 else "✗"
    print(f"  {flag} β={betas_naive[i]:.1f} ↔ β={betas_naive[i + 1]:.1f}: {float(r):.1%}")

The large gap between β=0.1 and β=0.3 gives near-zero swap acceptance — those chains never communicate, so the cold chain still can't escape its mode.

## Tuning the temperature ladder

We need acceptance rates in the **20–50% range on every pair**. The iterative refinement below starts from a coarse geometric ladder and repeatedly inserts a new temperature at the geometric midpoint of the worst-performing pair until convergence.

In [ ]:
def run_acceptance(betas, key, n_rounds=300):
    n = len(betas)
    ebms_t = [IsingEBM(nodes, edges, biases, weights, jnp.array(b)) for b in betas]
    progs_t = [IsingSamplingProgram(e, free_blocks, []) for e in ebms_t]
    key, subkey = jax.random.split(key)
    init_t = hinton_init(subkey, ebms_t[0], free_blocks, ())

    @jax.jit
    def _run(key, states):
        return parallel_tempering(key, ebms_t, progs_t, states, [], n_rounds=n_rounds, gibbs_steps_per_round=2)

    key, subkey = jax.random.split(key)
    _, _, stats = _run(subkey, [init_t] * n)
    return np.array(stats["acceptance_rate"])


def refine_ladder(betas, key, min_rate=0.2, max_iters=8):
    betas = sorted(list(betas))
    for i in range(max_iters):
        key, subkey = jax.random.split(key)
        rates = run_acceptance(betas, subkey)
        worst = int(np.argmin(rates))
        worst_rate = rates[worst]
        print(
            f"  Iter {i + 1}: {len(betas)} chains, "
            f"min rate = {worst_rate:.1%} at β={betas[worst]:.3f}↔{betas[worst + 1]:.3f}"
        )
        if worst_rate >= min_rate:
            print(f"  All pairs above {min_rate:.0%} — converged.")
            break
        new_beta = float(np.sqrt(betas[worst] * betas[worst + 1]))
        betas.insert(worst + 1, new_beta)
    return betas, rates


print("Refining from a 5-chain geometric ladder...")
start_betas = np.geomspace(0.1, beta_target, 5).tolist()
key, subkey = jax.random.split(key)
refined_betas, _ = refine_ladder(start_betas, subkey)
print(f"\nFinal ladder: {[f'{b:.3f}' for b in refined_betas]}")

In [ ]:
# Plot final acceptance rates
key, subkey = jax.random.split(key)
final_rates = run_acceptance(refined_betas, subkey)
pair_labels = [f"{refined_betas[i]:.2f}↔{refined_betas[i + 1]:.2f}" for i in range(len(refined_betas) - 1)]
bar_colors = ["green" if 0.2 <= r <= 0.5 else "orange" if r > 0.5 else "red" for r in final_rates]

plt.figure(figsize=(9, 3))
plt.bar(pair_labels, final_rates, color=bar_colors, edgecolor="white")
plt.axhline(0.2, color="k", lw=1, linestyle="--", label="20% lower target")
plt.axhline(0.5, color="k", lw=1, linestyle=":", label="50% upper target")
plt.ylabel("Acceptance rate")
plt.title(f"Refined ladder — {len(refined_betas)} chains")
plt.xticks(rotation=25, ha="right")
plt.legend()
plt.tight_layout()
plt.show()

## Comparison: naive vs tuned ladder

Now we can see the real benefit. With a well-tuned ladder, the cold chain mixes freely between both modes.

In [ ]:
# Run PT with refined betas
ebms_refined = [IsingEBM(nodes, edges, biases, weights, jnp.array(b)) for b in refined_betas]
programs_refined = [IsingSamplingProgram(e, free_blocks, []) for e in ebms_refined]
n_ref = len(refined_betas)

key, subkey = jax.random.split(key)
init_ref = hinton_init(subkey, ebms_refined[0], free_blocks, ())


@jax.jit
def one_round_refined(key, states):
    return parallel_tempering(
        key,
        ebms_refined,
        programs_refined,
        states,
        [],
        n_rounds=1,
        gibbs_steps_per_round=2,
    )


mag_refined = []
states_ref = [init_ref] * n_ref
key, subkey = jax.random.split(key)
for k in jax.random.split(subkey, 400):
    states_ref, _, _ = one_round_refined(k, states_ref)
    cold = jnp.concatenate(states_ref[-1])
    mag_refined.append(float(jnp.mean(2 * cold.astype(jnp.int8) - 1)))

# Three-way comparison
fig, axes = plt.subplots(3, 1, figsize=(12, 7), sharex=True)

for ax, mag, color, title in zip(
    axes,
    [mag_gibbs, mag_naive, mag_refined],
    ["steelblue", "orangered", "green"],
    [
        "Standard Gibbs — trapped",
        f"PT with naive betas {[round(b, 1) for b in betas_naive]} — still trapped",
        f"PT with refined betas ({len(refined_betas)} chains) — mixing between modes",
    ],
):
    ax.plot(mag, lw=0.8, color=color)
    ax.axhline(0, color="k", lw=0.5, linestyle="--")
    ax.set_ylim(-1.1, 1.1)
    ax.set_ylabel("Magnetization")
    ax.set_title(title)

axes[-1].set_xlabel("Round")
plt.tight_layout()
plt.show()

## Performance: compile time stays flat with more chains

In the original thrml, more chains meant longer compile time because each chain was unrolled separately into the XLA graph. With `jax.vmap` in THRML-Boost, compile time is essentially constant — you only pay in runtime, which scales linearly.

In [ ]:
chain_counts = [2, 4, 8, 16]
compile_times = []

for n in chain_counts:
    betas_n = np.geomspace(0.1, beta_target, n).tolist()
    ebms_n = [IsingEBM(nodes, edges, biases, weights, jnp.array(b)) for b in betas_n]
    progs_n = [IsingSamplingProgram(e, free_blocks, []) for e in ebms_n]
    key, subkey = jax.random.split(key)
    init_n = hinton_init(subkey, ebms_n[0], free_blocks, ())

    @jax.jit
    def _run(key, states):
        return parallel_tempering(key, ebms_n, progs_n, states, [], n_rounds=5, gibbs_steps_per_round=1)

    key, subkey = jax.random.split(key)
    t0 = time.time()
    result = _run(subkey, [init_n] * n)
    jax.block_until_ready(result[0])
    compile_times.append(time.time() - t0)
    print(f"  {n:2d} chains: {compile_times[-1]:.2f}s")

plt.figure(figsize=(6, 3))
plt.plot(chain_counts, compile_times, "o-", color="steelblue", label="THRML-Boost (vmap)")
plt.xlabel("Number of chains")
plt.ylabel("First-call time (compile + run, s)")
plt.title("Compile time is flat across chain counts")
plt.legend()
plt.tight_layout()
plt.show()

## Summary

Parallel tempering in THRML-Boost:

- Escapes multimodal traps that standard Gibbs cannot
- All chains run in a single `jax.vmap` kernel — no compile cost for extra chains
- Swap statistics guide temperature ladder tuning
- The iterative refinement loop above handles problem-specific structure automatically

```python
final_states, sampler_states, stats = parallel_tempering(
    key,
    ebms,            # one IsingEBM per temperature
    programs,        # one IsingSamplingProgram per temperature
    init_states,     # one initial state per temperature
    clamp_state=[],
    n_rounds=500,
    gibbs_steps_per_round=2,
)

# Check your ladder
print(stats['acceptance_rate'])   # aim for 0.2–0.5 on every pair
```